In [1]:
bad = ["Venture Capital", "legal practices", "consulting groups", "LLC (check for name)"]
good =  ["Biotech", "crypto", "AI", "tech", "medtech", "pharmaceuticals", "med devices", if a company has already raised venture capital funding, low amount of employees (look into funding round and number of employees), 18-24 months post last round]

In [4]:
# Query explainsion
# Making vectors
# Getting Score
# classifying


# Just using ChatGPT

In [6]:
import spacy
from nltk.corpus import wordnet

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

bad_list = ["venture capital", "legal practices", "consulting groups", "LLC"]
good_list = ["biotech", "crypto", "AI", "tech", "medtech", "pharmaceuticals", "medical devices"]

def get_word_synonyms(word):
    doc = nlp(word)
    synonyms = set()
    for token in doc:
        synonyms |= get_synonyms(token.text)
    return synonyms

print("Synonyms for words in the bad list:")
for word in bad_list:
    synonyms = get_word_synonyms(word)
    print(f"{word}: {synonyms}")

print("\nSynonyms for words in the good list:")
for word in good_list:
    synonyms = get_word_synonyms(word)
    print(f"{word}: {synonyms}")


Synonyms for words in the bad list:
venture capital: {'adventure', 'uppercase', 'working_capital', 'embark', 'upper-case_letter', 'chapiter', 'capital_letter', 'Das_Kapital', 'venture', 'jeopardize', 'pretend', 'Capital', 'cap', 'majuscule', 'great', 'hazard', 'stake', 'speculation', 'Washington', 'capital', 'guess'}
legal practices: {'rehearse', 'sound', 'practise', 'exercise', 'do', 'practice_session', 'use', 'effectual', 'legal', 'pattern', 'drill', 'apply', 'recitation', 'practice', 'praxis', 'commit'}
consulting groups: {'confabulate', 'confer', 'consult', 'aggroup', 'confer_with', 'refer', 'mathematical_group', 'group', 'chemical_group', 'radical', 'grouping', 'look_up', 'confab'}
LLC: set()

Synonyms for words in the good list:
biotech: {'biotechnology', 'biotech'}
crypto: set()
AI: {'AI', 'ai', 'artificial_insemination', 'three-toed_sloth', 'Army_Intelligence', 'artificial_intelligence', 'Bradypus_tridactylus'}
tech: {'tech', 'technical_school'}
medtech: set()
pharmaceuticals: 

In [7]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# User query
query = "venture capital"

# Tokenize query
tokenized_query = tokenizer.encode(query, return_tensors='pt')

# Generate BERT embeddings
with torch.no_grad():
    outputs = model(tokenized_query)
    query_embedding = outputs[0].squeeze(0)  # CLS token embedding

# Example corpus
corpus = ["startup funding", "angel investment", "venture capitalist", "entrepreneurship"]

# Tokenize and embed corpus
tokenized_corpus = tokenizer(corpus, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**tokenized_corpus)
    corpus_embeddings = outputs[0][:, 0, :].numpy()  # CLS token embeddings

# Calculate cosine similarity between query embedding and corpus embeddings
similarity_scores = cosine_similarity(query_embedding.numpy(), corpus_embeddings)

# Sort corpus by similarity scores
sorted_indices = np.argsort(similarity_scores[0])[::-1]

# Retrieve top similar terms
expanded_query = [corpus[i] for i in sorted_indices[:3]]  # Get top 3 similar terms
expanded_query = ' '.join(expanded_query)

print("Original query:", query)
print("Expanded query:", expanded_query)


C:\Users\rs659\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rs659\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Original query: venture capital
Expanded query: venture capitalist angel investment startup funding


In [12]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Original sentence
# original_sentence = "I'm interested in venture capital and legal practices."
original_sentence = "Public Good Pharma develops new low-cost therapies from system-wide cost savings of pharmaceutical payers  Our model employs 2 key mechanisms:   Interventional Pharmacoeconomic (IVPE) clinical trials generate guaranteed cost savings for payers by comparing in-use, expensive therapies to low-cost, safe and effective alternatives   +  Pay-For-Success Contracts (PFS) allow payers to incentivize regulatory approval for a low-cost alternative without taking risk. If IVPE trials are successful, PFS contracts guarantee supply.  "

# Lists of words to find similar terms for
bad_list = ["venture capital", "legal practices", "consulting groups", "LLC"]
good_list = ["biotech", "crypto", "AI", "tech", "medtech", "pharmaceuticals", "medical devices"]

# Tokenize the original sentence
tokenized_sentence = tokenizer.encode(original_sentence, return_tensors='pt')

# Generate BERT embeddings for the tokens in the sentence
with torch.no_grad():
    outputs = model(tokenized_sentence)
    sentence_embeddings = outputs[0].squeeze(0)  # CLS token embedding

# Function to get similar terms for a word from the provided list
def get_similar_terms(word, word_list):
    tokenized_words = tokenizer(word_list, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokenized_words)
        embeddings = outputs[0][:, 0, :].numpy()  # CLS token embeddings
    similarity_scores = cosine_similarity(sentence_embeddings.numpy(), embeddings)
    sorted_indices = np.argsort(similarity_scores[0])[::-1]
    similar_words = [word_list[i] for i in sorted_indices[:3]]  # Get top 3 similar terms
    return similar_words

# Expand the original sentence by replacing matched words with similar terms
expanded_sentence = ""
for token, word in zip(tokenized_sentence[0], tokenizer.convert_ids_to_tokens(tokenized_sentence[0])):
    if word in bad_list:
        similar_terms = get_similar_terms(word, bad_list)
        expanded_sentence += f"{{{' '.join(similar_terms)}}} "
    elif word in good_list:
        similar_terms = get_similar_terms(word, good_list)
        expanded_sentence += f"{{{' '.join(similar_terms)}}} "
    else:
        expanded_sentence += word + " "

print("Original sentence:", original_sentence)
print("Expanded sentence:", expanded_sentence)


Original sentence: Public Good Pharma develops new low-cost therapies from system-wide cost savings of pharmaceutical payers  Our model employs 2 key mechanisms:   Interventional Pharmacoeconomic (IVPE) clinical trials generate guaranteed cost savings for payers by comparing in-use, expensive therapies to low-cost, safe and effective alternatives   +  Pay-For-Success Contracts (PFS) allow payers to incentivize regulatory approval for a low-cost alternative without taking risk. If IVPE trials are successful, PFS contracts guarantee supply.  
Expanded sentence: [CLS] public good ph ##arm ##a develops new low - cost the ##ra ##pies from system - wide cost savings of pharmaceutical pay ##ers our model employs 2 key mechanisms : intervention ##al ph ##arm ##aco ##economic ( iv ##pe ) clinical trials generate guaranteed cost savings for pay ##ers by comparing in - use , expensive the ##ra ##pies to low - cost , safe and effective alternatives + pay - for - success contracts ( p ##fs ) allow 

In [13]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Original sentence
original_sentence = "I'm interested in venture capital and legal practices."

# Lists of words to find similar terms for
bad_list = ["venture capital", "legal practices", "consulting groups", "LLC"]
good_list = ["biotech", "crypto", "AI", "tech", "medtech", "pharmaceuticals", "medical devices"]

# Tokenize the original sentence
tokenized_sentence = tokenizer.encode(original_sentence, return_tensors='pt')

# Generate BERT embeddings for the tokens in the sentence
with torch.no_grad():
    outputs = model(tokenized_sentence)
    sentence_embeddings = outputs[0].squeeze(0)  # CLS token embedding

# Function to get similar terms for a word from the provided list
def get_similar_terms(word, word_list):
    tokenized_words = tokenizer(word_list, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokenized_words)
        embeddings = outputs[0][:, 0, :].numpy()  # CLS token embeddings
    similarity_scores = cosine_similarity(sentence_embeddings.numpy(), embeddings)
    sorted_indices = np.argsort(similarity_scores[0])[::-1]
    similar_words = [word_list[i] for i in sorted_indices[:3]]  # Get top 3 similar terms
    return similar_words

# Expand the original sentence by replacing matched words with similar terms
expanded_sentence = ""
for token, word in zip(tokenized_sentence[0], tokenizer.convert_ids_to_tokens(tokenized_sentence[0])):
    if word in bad_list:
        similar_terms = get_similar_terms(word, bad_list)
        expanded_sentence += f"{{{' '.join(similar_terms)}}} "
    elif word in good_list:
        similar_terms = get_similar_terms(word, good_list)
        expanded_sentence += f"{{{' '.join(similar_terms)}}} "
    else:
        expanded_sentence += f"{{{word}}} "

print("Original sentence:", original_sentence)
print("Expanded sentence:", expanded_sentence)


Original sentence: I'm interested in venture capital and legal practices.
Expanded sentence: {[CLS]} {i} {'} {m} {interested} {in} {venture} {capital} {and} {legal} {practices} {.} {[SEP]} 


In [14]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Original sentence
original_sentence = "I'm interested in venture capital and legal practices."

# Lists of words to find similar terms for
bad_list = ["venture capital", "legal practices", "consulting groups", "LLC"]
good_list = ["biotech", "crypto", "AI", "tech", "medtech", "pharmaceuticals", "medical devices"]

# Tokenize the original sentence
tokenized_sentence = tokenizer.encode(original_sentence, return_tensors='pt')

# Generate BERT embeddings for the tokens in the sentence
with torch.no_grad():
    outputs = model(tokenized_sentence)
    sentence_embeddings = outputs[0].squeeze(0)  # CLS token embedding

# Function to get similar terms for a word from the provided list
def get_similar_terms(word, word_list):
    tokenized_words = tokenizer(word_list, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokenized_words)
        embeddings = outputs[0][:, 0, :].numpy()  # CLS token embeddings
    similarity_scores = cosine_similarity(sentence_embeddings.numpy(), embeddings)
    sorted_indices = np.argsort(similarity_scores[0])[::-1]
    similar_words = [word_list[i] for i in sorted_indices[:3]]  # Get top 3 similar terms
    return similar_words

# Expand the original sentence by replacing matched words with similar terms
expanded_sentence = ""
for token, word in zip(tokenized_sentence[0], tokenizer.convert_ids_to_tokens(tokenized_sentence[0])):
    if word in bad_list:
        similar_terms = get_similar_terms(word, bad_list)
        expanded_sentence += f"{{{ ' '.join(similar_terms) }}} "
    elif word in good_list:
        similar_terms = get_similar_terms(word, good_list)
        expanded_sentence += f"{{{ ' '.join(similar_terms) }}} "
    else:
        expanded_sentence += f"{word} "

print("Original sentence:", original_sentence)
print("Expanded sentence:", expanded_sentence)


Original sentence: I'm interested in venture capital and legal practices.
Expanded sentence: [CLS] i ' m interested in venture capital and legal practices . [SEP] 


In [ ]:
api_key = "sk2PFfoB2vewe6gki6tsm95yEsE5Ry6X"